## Start 

In [96]:
from pyspark.sql import SparkSession

#Initialize the session 
spark=SparkSession.builder.appName("Flight Price Prediction").getOrCreate()

#Read the data
data="./data/flights_data_.csv" #Chemin vers le fichier
df=spark.read.csv(data,header=True,inferSchema=True)


In [97]:
#Show the data
df.show()
#Number of rows
df.count()

+--------------------+-----------+-----+--------+-----------+----------+
|            Airlines|Total_Stops|Price|Duration|Destination|      Date|
+--------------------+-----------+-----+--------+-----------+----------+
|            Tunisair|          0| 81 €|2h 15min|        MAD|2025-06-29|
|            Tunisair|          0| 81 €|2h 15min|        MAD|2025-06-29|
|          Air Europa|          0| 83 €|2h 15min|        MAD|2025-06-29|
|     Vueling, Iberia|          1|102 €|5h 20min|        MAD|2025-06-29|
|    Tunisair, Iberia|          1|105 €|5h 55min|        MAD|2025-06-29|
|     Aegean Airlines|          1|117 €|9h 35min|        MAD|2025-06-29|
|    Tunisair, Iberia|          1|137 €|4h 45min|        MAD|2025-06-29|
|         ITA Airways|          1|177 €|4h 49min|        MAD|2025-06-29|
|   Tunisair, Ryanair|          1|110 €|6h 30min|        MAD|2025-06-29|
|    Transavia France|          1|137 €|7h 05min|        MAD|2025-06-29|
|   Nouvelair, Iberia|          1|119 €|6h 15min|  

7394

## Clean the data 

In [98]:
df.printSchema()
# #Data format 
#  |-- Airlines: string (nullable = true)
#  |-- Total_Stops: integer (nullable = true)
#  |-- Price: string (nullable = true)
#  |-- Duration: string (nullable = true)
#  |-- Destination: string (nullable = true)
#  |-- Date: date (nullable = true)
#It is neccesary to change few format

root
 |-- Airlines: string (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Price: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Date: date (nullable = true)



In [99]:
from pyspark.sql.functions import (
    col,  
    to_timestamp, 
    when, 
    split,
   regexp_replace,lit,
date_format
)


# 1. Convert Date 
df = df.withColumn("Date",date_format( to_timestamp(col("Date"), "yyyy-MM-dd"),"yyyy-MM-dd"))

# 2.Convert price to integer
df = df.withColumn("Price", regexp_replace(col("Price"), "\xa0", " "))

df = df.withColumn("Price", split(col("Price"), " ")[0].cast("integer"))





df.printSchema()
df.show()

root
 |-- Airlines: string (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Date: string (nullable = true)

+--------------------+-----------+-----+--------+-----------+----------+
|            Airlines|Total_Stops|Price|Duration|Destination|      Date|
+--------------------+-----------+-----+--------+-----------+----------+
|            Tunisair|          0|   81|2h 15min|        MAD|2025-06-29|
|            Tunisair|          0|   81|2h 15min|        MAD|2025-06-29|
|          Air Europa|          0|   83|2h 15min|        MAD|2025-06-29|
|     Vueling, Iberia|          1|  102|5h 20min|        MAD|2025-06-29|
|    Tunisair, Iberia|          1|  105|5h 55min|        MAD|2025-06-29|
|     Aegean Airlines|          1|  117|9h 35min|        MAD|2025-06-29|
|    Tunisair, Iberia|          1|  137|4h 45min|        MAD|2025-06-29|
|         ITA Airways| 

In [100]:
df = df.withColumn("Airlines", regexp_replace(col("Airlines"), "\xa0", " "))
df=df.withColumn("Airlines_list",split(col("Airlines"), ", "))
df.printSchema()
df.show()

root
 |-- Airlines: string (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Airlines_list: array (nullable = true)
 |    |-- element: string (containsNull = false)

+--------------------+-----------+-----+--------+-----------+----------+--------------------+
|            Airlines|Total_Stops|Price|Duration|Destination|      Date|       Airlines_list|
+--------------------+-----------+-----+--------+-----------+----------+--------------------+
|            Tunisair|          0|   81|2h 15min|        MAD|2025-06-29|          [Tunisair]|
|            Tunisair|          0|   81|2h 15min|        MAD|2025-06-29|          [Tunisair]|
|          Air Europa|          0|   83|2h 15min|        MAD|2025-06-29|        [Air Europa]|
|     Vueling, Iberia|          1|  102|5h 20min|        MAD|2025-06-29|   [Vueling, Iberia]|
| 

In [101]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType

# Convert duration from 'Xh Ym' format to hours
def duration_to_hours(duration):
    """
    Convert flight duration to hours.
    Args:
        duration (str): Duration in format 'Xh Ym' or 'Xh' or 'Ym'.
    Returns:
        float: Total duration in hours.
    """
    if not duration:
        return None
    
    total_hours = 0.0
    
    # Process hours and minutes
    if "h" in duration:
        hours = float(duration.split("h")[0].strip())
        total_hours += hours  # Add hours directly
        if "m" in duration:
            minutes = int(duration.split("h")[1].split("m")[0].strip())
            total_hours += minutes / 60.0  # Convert minutes to hours and add
    elif "m" in duration:
        minutes = int(duration.replace("m", "").strip())
        total_hours += minutes / 60.0  # Convert minutes to hours
    
    return total_hours

# Create a UDF to apply the function
duration_to_hours_udf = udf(duration_to_hours, FloatType())

# Apply the transformation on the DataFrame to convert duration to hours
df = df.withColumn("Duration", duration_to_hours_udf(col("Duration")))

# Show the schema and the DataFrame data
df.printSchema()
df.show(truncate=False)


root
 |-- Airlines: string (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Duration: float (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Airlines_list: array (nullable = true)
 |    |-- element: string (containsNull = false)

+--------------------------+-----------+-----+---------+-----------+----------+----------------------------+
|Airlines                  |Total_Stops|Price|Duration |Destination|Date      |Airlines_list               |
+--------------------------+-----------+-----+---------+-----------+----------+----------------------------+
|Tunisair                  |0          |81   |2.25     |MAD        |2025-06-29|[Tunisair]                  |
|Tunisair                  |0          |81   |2.25     |MAD        |2025-06-29|[Tunisair]                  |
|Air Europa                |0          |83   |2.25     |MAD        |2025-06-29|[Air Europa]                |
|Vuelin

In [102]:
from pyspark.sql.types import StringType


def destination_converter(destination):
    destinations={"TUN":"Tunis","CRL":"Brussels","ANR":"Antwerp","OST":"Ostrava","LUX":"Luxembourg","NDJ":"N'Djamena","ABJ":"Abidjan","PAR":"Paris","ALG":"Alger","COO":"Cotonou","MAD":"Madrid"}
    return destinations.get(destination)

destination_converter_udf = udf(destination_converter, StringType())



df = df.withColumn("Destination", destination_converter_udf(col("Destination")))
df = df.withColumn("Depart",lit("Tunis"))
df.printSchema()
df.show(truncate=False)
    

root
 |-- Airlines: string (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Duration: float (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Airlines_list: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Depart: string (nullable = false)

+--------------------------+-----------+-----+---------+-----------+----------+----------------------------+------+
|Airlines                  |Total_Stops|Price|Duration |Destination|Date      |Airlines_list               |Depart|
+--------------------------+-----------+-----+---------+-----------+----------+----------------------------+------+
|Tunisair                  |0          |81   |2.25     |Madrid     |2025-06-29|[Tunisair]                  |Tunis |
|Tunisair                  |0          |81   |2.25     |Madrid     |2025-06-29|[Tunisair]                  |Tunis |
|Air Europa                |0          |83

In [103]:

# Remove rows with null values in critical columns
df = df.filter(
    (col("Duration").isNotNull()) &
    (col("Price").isNotNull()) &
    (col("Airlines").isNotNull()) &
    (col("Date").isNotNull())
)


df.printSchema()


root
 |-- Airlines: string (nullable = true)
 |-- Total_Stops: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Duration: float (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Airlines_list: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Depart: string (nullable = false)



In [104]:
df.show()

+--------------------+-----------+-----+---------+-----------+----------+--------------------+------+
|            Airlines|Total_Stops|Price| Duration|Destination|      Date|       Airlines_list|Depart|
+--------------------+-----------+-----+---------+-----------+----------+--------------------+------+
|            Tunisair|          0|   81|     2.25|     Madrid|2025-06-29|          [Tunisair]| Tunis|
|            Tunisair|          0|   81|     2.25|     Madrid|2025-06-29|          [Tunisair]| Tunis|
|          Air Europa|          0|   83|     2.25|     Madrid|2025-06-29|        [Air Europa]| Tunis|
|     Vueling, Iberia|          1|  102|5.3333335|     Madrid|2025-06-29|   [Vueling, Iberia]| Tunis|
|    Tunisair, Iberia|          1|  105|5.9166665|     Madrid|2025-06-29|  [Tunisair, Iberia]| Tunis|
|     Aegean Airlines|          1|  117| 9.583333|     Madrid|2025-06-29|   [Aegean Airlines]| Tunis|
|    Tunisair, Iberia|          1|  137|     4.75|     Madrid|2025-06-29|  [Tunisa

In [105]:
#Delete duplicates
df = df.distinct()
df.count()

5707